# **DB 생성 및 관리**

## **1.환경준비**

In [1]:
import pandas as pd
import sqlite3

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## **2.DB 작업**

### (1) DB 생성 및 연결

* 데이터베이스 연결
    * 폴더는 미리 생성
    * 해당 파일이 없으면 새로 생성, 있으면 연결

In [2]:
path = 'testdb.db'
conn = sqlite3.connect(path)

### (2) DB 작업 절차

* 1) 커서 활용 : 대부분의 SQL 작업 가능
    * ① 연결
    * ② 커서 선언
    * ③ 작업 : sql
    * ④ 커밋(수정작업 저장)
    * ⑤ 연결종료

* 2) pandas 함수 활용 : 주로 insert, update, delete, select, create table & insert
    * ① 연결
    * ② 작업 : pandas 함수 이용
    * ③ 연결종료

#### 1) 테이블 생성
* 테이블 생성 작업
    * DB Browser for SQLite 에서 직접 수행
    * 파이썬에서 **연결**하고, **커서**를 생성해서 작업

In [3]:
# 커서 객체 생성
cursor = conn.cursor()

# test 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS test (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER NOT NULL
)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

#### 2) insert
* 데이터프레임에서 직접 DB로 보내기 : df.to_sql

In [4]:
# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : to_sql
data = pd.DataFrame({'name': ['Alice', 'Bob', 'Charlie', 'David'], 'age': [30, 25, 35, 40]})
data.to_sql('test', conn, if_exists='append', index=False) # test 테이블이 있으면 insert, 없으면 생성

# ③ 연결 종료
conn.close()

#### 3) Select

In [5]:
# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : select
df = pd.read_sql('SELECT * FROM test', conn)
display(df)

# ③ 연결 종료
conn.close()

,id,name,age
0,1,Alice,30
1,2,Bob,25
2,3,Charlie,35
3,4,David,40


#### 4) Drop

* 커서로만 작업 가능

In [6]:
# ① 연결
conn = sqlite3.connect(path)

# ② 커서 선언
cursor = conn.cursor()

# ③ 작업 : 테이블 삭제
cursor.execute('DROP TABLE IF EXISTS test')

# ④ 커밋(수정작업 저장)
conn.commit()

# ⑤ 연결종료
conn.close()

## **3.관리용 DB**


### (1) DB 생성

* DB를 아래 경로와 이름으로 생성
    * project7/db/em.db

In [7]:
path = 'db/em.db'
conn = sqlite3.connect(path)

### (2) log 테이블 생성
* 칼럼 구성

|column name | descrition | data type | comment |
|----|----|----|----|
|datetime | 요청일시 | text | yyyy-mm-dd hh:mi:ss |
|input_text | 입력text | text |  |
|input_latitude | 입력 위도 | real |  |
|input_longitude | 입력 경도 | real |  |
|em_class | 응급 등급(1~5) | integer |  |
|hospital1 | 추천병원명 | text |  |
|addr1 | 추천병원주소 | text |  |
|tel1 | 추천병원전화번호 | text |  |
|hospital2 | 추천병원명2 | text |  |
|addr2 | 추천병원주소2 | text |  |
|tel2 | 추천병원전화번호2 | text |  |
|hospital3 | 추천병원명3 | text |  |
|addr3 | 추천병원주소3 | text |  |
|tel3 | 추천병원전화번호3 | text |  |

In [8]:
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS log (
    id INTEGER PRIMARY KEY,
    datetime TEXT,
    input_text TEXT,
    input_latitude REAL,
    input_longitude REAL,
    em_class INTEGER,
    hospital1 TEXT,
    addr1 TEXT,
    tel1 TEXT,
    hospital2 TEXT,
    addr2 TEXT,
    tel2 TEXT,
    hospital3 TEXT,
    addr3 TEXT,
    tel3 TEXT
)
''')


# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

In [9]:
conn = sqlite3.connect(path)

# ② 작업 : select
df = pd.read_sql('SELECT * FROM log', conn)
display(df)

# ③ 연결 종료
conn.close()

,id,datetime,input_text,input_latitude,input_longitude,em_class,hospital1,addr1,tel1,hospital2,addr2,tel2,hospital3,addr3,tel3


### (3) log 테이블 입력 함수

* 현재 일시 저장하기 예제
            # 현재 연월일 시분초 뽑기
            from datetime import datetime
            dt = datetime.now()
            dt = dt.strftime('%Y-%m-%d %H:%M:%S')


In [10]:
from datetime import datetime

In [11]:
# conn = sqlite3.connect(path)

# dt = datetime.now()
# dt = dt.strftime('%Y-%m-%d %H:%M:%S')

# data = pd.DataFrame({'datetime': [dt]})
# data.to_sql('log', conn, if_exists='append', index=False)

# conn.close()

In [12]:
conn = sqlite3.connect(path)

df = pd.read_sql('SELECT * FROM log', conn)
display(df)

conn.close()

,id,datetime,input_text,input_latitude,input_longitude,em_class,hospital1,addr1,tel1,hospital2,addr2,tel2,hospital3,addr3,tel3


In [13]:
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification

import emergency_ai26 as em

from datetime import datetime

c:\Users\User\anaconda3\envs\mini7-fastapi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
path = './'

dt = datetime.now()
dt = dt.strftime('%Y-%m-%d %H:%M:%S')

data = pd.DataFrame({'datetime': [dt]})
# data.to_sql('log', conn, if_exists='append', index=False)

openai.api_key = em.load_keys(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

map_key = em.load_keys(path + 'map_key.txt')
map_key = json.loads(map_key)
c_id, c_key = map_key['c_id'], map_key['c_key']

emergency = pd.read_csv(path + '응급실 정보.csv')

# 모델, 토크나이저 로드
save_directory = path + "fine_tuned_bert_ai26"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)


# 2. 데이터 처리(pipeline) ---------------------------------
# input
audio_path = path + 'audio/'
filename = 'audio1.mp3'

# 처리
result = em.audio2text(audio_path, filename)
result = em.text2summary(result)

data["input_text"] = result

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
predicted_class, _ = em.predict(result, model, tokenizer,device)

data["em_class"] = predicted_class

# 테스트 input --------------------
predicted_class = 2  # 테스트용
start_lat, start_lng = 35.548238, 129.307011
# ---------------------------------

data[["input_latitude","input_longitude"]] = [start_lat,start_lng]

if predicted_class <= 2 :
    result = em.recommend_hospital(emergency, start_lat, start_lng, 0.1, c_id, c_key)
    display(result)
    for idx, row in result.iterrows():
        data[f"hospital{idx+1}"] = row["병원이름"]
        data[f"addr{idx+1}"] = row["주소"]
        data[f"tel{idx+1}"] = row["전화번호 1"]
    
else :
    print('개인 건강관리')
    
path = 'db/em.db'
conn = sqlite3.connect(path)

data.to_sql('log', conn, if_exists='append', index=False) # test 테이블이 있으면 insert, 없으면 생성

conn.close()

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,거리
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011,0
1,의료법인동강의료재단동강병원,울산광역시 중구 태화로 239 (태화동),지역응급의료센터,052-241-1114,052-241-1190,35.553410,129.301943,2032
2,의료법인정안의료재단중앙병원,"울산광역시 남구 문수로 472, 중앙병원 (신정동)",지역응급의료센터,052-226-1100,052-226-1119,35.532088,129.304840,2125


In [15]:
conn = sqlite3.connect(path)

df = pd.read_sql('SELECT * FROM log', conn)
display(df)

conn.close()

,id,datetime,input_text,input_latitude,input_longitude,em_class,hospital1,addr1,tel1,hospital2,addr2,tel2,hospital3,addr3,tel3
0,1,2024-12-19 15:50:28,남성이 머리에서 피가 나면서 넘어져서 어지럽다고 합니다. 현재 물을 마셔서 있지만 ...,35.548238,129.307011,3,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),052-220-3300,의료법인동강의료재단동강병원,울산광역시 중구 태화로 239 (태화동),052-241-1114,의료법인정안의료재단중앙병원,"울산광역시 남구 문수로 472, 중앙병원 (신정동)",052-226-1100
